<a href="https://colab.research.google.com/github/krithikajagannathan/dog-vs-cat-classifier/blob/main/dogs_vs_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!kaggle datasets download -d salader/dogs-vs-cats


Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

with zipfile.ZipFile('dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall('data')


In [ ]:
import os
import shutil

# Paths for the dataset
base_dir = '/content/dogs_vs_cats'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Create directories if they do not exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Assume you have directories /content/dogs_vs_cats/train and /content/dogs_vs_cats/test
# If the structure is different, you may need to adjust accordingly


In [ ]:
!pip install tensorflow keras


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/dogs_vs_cats/train',  # Corrected path
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/dogs_vs_cats/train',  # Corrected path
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Train the model
model.fit(
    train_generator,
    epochs=10,  # Use a smaller number of epochs for quick iteration
    validation_data=validation_generator
)

# Save the model
model.save('/content/cats_vs_dogs_model.h5')



9406464/9406464 [==============================] - 0s 0us/step
Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Epoch 1/10
500/500 [==============================] - 403s 798ms/step - loss: 0.1432 - accuracy: 0.9469 - val_loss: 0.0964 - val_accuracy: 0.9572
Epoch 2/10
500/500 [==============================] - 384s 768ms/step - loss: 0.1035 - accuracy: 0.9598 - val_loss: 0.0944 - val_accuracy: 0.9600
Epoch 3/10
500/500 [==============================] - 403s 806ms/step - loss: 0.0907 - accuracy: 0.9650 - val_loss: 0.0898 - val_accuracy: 0.9630
Epoch 4/10
500/500 [==============================] - 377s 754ms/step - loss: 0.0836 - accuracy: 0.9669 - val_loss: 0.0900 - val_accuracy: 0.9613
Epoch 5/10
500/500 [==============================] - 395s 790ms/step - loss: 0.0788 - accuracy: 0.9691 - val_loss: 0.0883 - val_accuracy: 0.9647
Epoch 6/10
500/500 [==============================] - 391s 783ms/step - loss: 0.0694 - accuracy: 0.9722 - val_loss: 0.0905

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


125/125 [==============================] - 77s 616ms/step - loss: 0.0987 - accuracy: 0.9657
Validation Loss: 0.09866145253181458
Validation Accuracy: 0.965749979019165


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_image(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        return 'dog'
    else:
        return 'cat'

# Example usage
img_path = '/content/pexels-pixabay-45201.jpg'  # Replace with the path to your image
print(predict_image(img_path))


1/1 [==============================] - 1s 1s/step
cat


In [ ]:
# Save the model
model.save('/content/cats_vs_dogs_model.h5')

# Download the model file
from google.colab import files
files.download('/content/cats_vs_dogs_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>